<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
function [V_car_now,dvdt_now,Omega_m_now] = EngineModel(engine_parameters,V_car)
    %dvdt_now,Omega_m_now,T_limit_now,F_wheel_now
    % unpack engine/car parameters
    r = engine_parameters(1); % wheel radius inches to m 
    ni =  engine_parameters(2); % in tooth
    no =  engine_parameters(3); % outer tooth 
    J_m =  engine_parameters(4); %0.023; % kg m^2 motor inertia 
    J_l =  engine_parameters(5); % load inertia  
    m_car = engine_parameters(6); %pound to kg
    Cd = engine_parameters(7); % assume coefficient of drag is 1
    A =  engine_parameters(8); %ft^2 to m^2
    Cr =  engine_parameters(9); % assume the rolling resistance of 4 wheels is 0.03 from published value of car wheels on Asphalt 
    omega_maximum = engine_parameters(10); %rpm 
    Tmax = engine_parameters(11); %from data sheet 
    P_max = engine_parameters(12); % W  battery provides 300 Amps/ Open circuit voltage around 100V/ assume resistance in battery around 0.05 ohms/ with 90 percent efficiency
    h = 0.01;
    % Calculations based on parameters or constants
    n_gear = ni/no; % gear ratio 
    p = 1.21; %kg/m^3 density of air 
    J_t = n_gear^2*J_l + J_m; % total inertia equation from website
    g = 9.81; % m/s^2 gravity 
    Tf = 0; % N*m torque due to internal system friction  
    mu = 0.7; % assume friction coefficient of tire equal to 1 (need data from tire)
    F_slip = mu*m_car*g; % mu*N
    T_slip = F_slip * r*n_gear; % maximum torque applied to wheels before tire slipping
    
    % identify T_limit (when the wheels start to slip)
    if T_slip <= Tmax 
        T_limit = T_slip; % maximum torque before wheel slipping
    end
    if T_slip > Tmax
        T_limit = Tmax; % maximum torque from motor 
    end
    omega_limit = 9.5488 * P_max /T_limit; % % Torque from the motor exceeds Torque limit until omega_motor reaches certain rpm 
    if V_car <= omega_maximum*2*pi/60*r*n_gear % before reaching maximum 6000 rpm
        
        if V_car <  omega_limit*2*pi/60*r*n_gear %m/s Torque from the motor exceeds maximum of 140 NM until reaches this speed 
            Tm = T_limit;
        end
        if V_car > omega_limit*2*pi/60*r*n_gear % m/s 
            Tm = P_max/(V_car(i)/n_gear/r); %  N*m
        end
        
        dVdt = 1/(J_m/(n_gear*r)+n_gear*r*m_car)*Tm -Tf - n_gear*r*0.5*Cd*p*A/(J_m/(n_gear*r)+n_gear*r*m_car)*V_car.^2 - n_gear*r*Cr*m_car*g /(J_m/(n_gear*r)+n_gear*r*m_car);
    
    else %V_car > omega_maximum*2*pi/60*r*n_gear % RPM from the motor reaches Maximum 6000 rpm 
        dVdt=0;
    end
    
    Omega_wheel = V_car/2/pi/r*60; % rpm
    Omega_m = V_car/n_gear/r*60/2/pi; %rpm
    V_car = V_car + dVdt*h;
    F_wheel = m_car*dVdt + 0.5*Cd*p*A*V_car.^2+Cr*m_car*g;
    
    % Assign output variables:
    T_limit_now = T_limit;
    V_car_now = V_car;
    F_wheel_now = F_wheel; 
    Omega_m_now = Omega_m;
    dvdt_now = dVdt
endfunction

In [2]:
r = 10.05/ 39.37; % wheel radius inches to m 
ni = 14;
no = 40;
J_m = 0.023; % kg m^2 motor inertia 
J_l = 0; % load inertia 
%J_t = n_gear^2*J_l + J_m; % total inertia equation from website 
m_car = 1000/ 2.205; %pound to kg
C = 1; % assume coefficient of drag is 1
p = 1.21; %kg/m^3 density of air 
A = 8/10.764; %ft^2 to m^2
Cr = 0.03; % assume the rolling resistance of 4 wheels is 0.03 from published value of car wheels on Asphalt 
ag = 9.81; % m/s^2 gravity 
Tf = 0; % N*m torque due to internal system friction  
mu = 0.7; % assume friction coefficient of tire equal to 1 (need data from tire)
%F_slip = mu*m_car*ag; % mu*N
%T_slip = F_slip * r*n_gear; % maximum torque applied to wheels before tire slipping
omega_maximum = 6000; %rpm 
Tmax = 140; %from data sheet 
P_max = 22500; % W  battery provides 300 Amps/ Open circuit voltage around 100V/ assume resistance in battery around 0.05 ohms/ with 90 percent efficiency to motor 
engine_parameters = [r,ni,no,J_m,J_l,m_car,C,A,Cr,omega_maximum,Tmax,P_max];
V_car = 5;
[V_car_now,dv_dt_now,Omega_m_now] = EngineModel(engine_parameters,V_car)
%,T_limit_now,F_wheel_now

dvdt_now =  3.1163
V_car_now =  5.0312
dv_dt_now =  3.1163
Omega_m_now =  534.41


In [3]:
EngineModel(engine_parameters,V_car)

dvdt_now =  3.1163
ans =  5.0312
